# EJERCICIO 1

### El propósito de este ejercicio es identificar endogeneidad en nuestro modelo, para ello utilizaremos la prueba de Hausman.


#### 1.- Revisamos la correlación entre  la variable *"índice de protección contra la expropiación ($avexpr_i$)" y los errores ($u_i$)*

Esta prueba tiene el objetivo de identificar la existencia de endogenidad. Para ello proponemos la siguiente estructura de pruebas de hipótesis:
\begin{eqnarray*}
    H_0 &:& Cov(avexpr_i, u_i) = 0 \text{ no hay endogeneidad} \\
    H_1 &:& Cov(avexpr_i, u_i) \neq 0 \text{ endogeneidad}
\end{eqnarray*}

En esta tarea deberá implemantar la prueba.

La prueba de Hausman se corre en dos etapas:


In [1]:
#importamos las librerías necesarias
#!pip install linearmodels==1.0
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm

# Para realizar este ejercicio utilice el conjunto de datos 'maketable4.dta'

In [2]:
# Impoprtación de los datos:
df4 = pd.read_stata('maketable4.dta')
df4

,shortnam,africa,lat_abst,rich4,avexpr,logpgp95,logem4,asia,loghjypl,baseco
0,AFG,0.0,0.366667,0.0,NaN,NaN,4.540098,1.0,NaN,NaN
1,AGO,1.0,0.136667,0.0,5.363636,7.770645,5.634789,0.0,-3.411248,1.0
2,ARE,0.0,0.266667,0.0,7.181818,9.804219,NaN,1.0,NaN,NaN
3,ARG,0.0,0.377778,0.0,6.386364,9.133459,4.232656,0.0,-0.872274,1.0
4,ARM,0.0,0.444444,0.0,NaN,7.682482,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
158,YUG,0.0,0.488889,0.0,6.318182,NaN,NaN,0.0,-1.203973,NaN
159,ZAF,1.0,0.322222,0.0,6.863636,8.885994,2.740840,0.0,-1.386294,1.0
160,ZAR,1.0,0.000000,0.0,3.500000,6.866933,5.480639,0.0,-3.411248,1.0
161,ZMB,1.0,0.166667,0.0,6.636364,6.813445,NaN,0.0,-2.975930,NaN


 ##### Añade la constante

In [3]:
df4['const'] = 1
df4.head()

,shortnam,africa,lat_abst,rich4,avexpr,logpgp95,logem4,asia,loghjypl,baseco,const
0,AFG,0.0,0.366667,0.0,NaN,NaN,4.540098,1.0,NaN,NaN,1
1,AGO,1.0,0.136667,0.0,5.363636,7.770645,5.634789,0.0,-3.411248,1.0,1
2,ARE,0.0,0.266667,0.0,7.181818,9.804219,NaN,1.0,NaN,NaN,1
3,ARG,0.0,0.377778,0.0,6.386364,9.133459,4.232656,0.0,-0.872274,1.0,1
4,ARM,0.0,0.444444,0.0,NaN,7.682482,NaN,1.0,NaN,NaN,1


#  1a etapa:

Corremos una regresión entre *índice de protección contra la expropiación ($avexpr$) y el instrumento tasa de mortalidad de los colonizadores ($logem4$)

In [11]:
# Filtrado de  datos

df4 = df4[df4['baseco'] == 1]



In [12]:
# Defina la vaeriable Endogena:
Endogena = df4['avexpr']

# defina las variables Exogenas (+ una constante)
Exogena = df4[['const', 'logem4']]

# Ejecute la regresión
reg1 = sm.OLS(endog = Endogena,
              exog = Exogena,
              missing='drop')
results=reg1.fit()

In [13]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 avexpr   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     22.95
Date:                Sat, 02 May 2020   Prob (F-statistic):           1.08e-05
Time:                        16:56:48   Log-Likelihood:                -104.83
No. Observations:                  64   AIC:                             213.7
Df Residuals:                      62   BIC:                             218.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.3414      0.611     15.296      0.0

# 2a etapa:

Recuperamos los residuales  $ \hat{u}_i $

In [14]:
# Creamos una nueva variable con residuales
df4['resid'] = results.resid
df4['predicted_avexpr'] = results.predict()

####  Incluímos los residuales anteriores en la regresión original que relaciona: *'logpgp95: lgaritmo del pib per cápita en 1995 y avexpr:índice de protección contra la expropiación 
$$
logpgp95_i = \beta_0 + \beta_1 avexpr_i + \alpha \hat{u}_i + u_i
$$

In [15]:
# Defina variables X y Y:

X = df4[['const', 'resid', 'predicted_avexpr']]

Y =  df4['logpgp95']

# Ejecutemos regresión
reg2 = sm.OLS(endog = Y,
              exog = X,
              missing='drop').fit()

#Imprimimos resultados
print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:               logpgp95   R-squared:                       0.671
Model:                            OLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     62.15
Date:                Sat, 02 May 2020   Prob (F-statistic):           1.91e-15
Time:                        16:56:54   Log-Likelihood:                -57.468
No. Observations:                  64   AIC:                             120.9
Df Residuals:                      61   BIC:                             127.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                1.9097      0.659  

# Dado el valor p-value de los residuales, ¿qué podemos decir sobre la endogeneidad del modelo dada  avexpr y logpgp95 ?

Discuta brevemente.
Debido a que el p value de los residuales resulto ser estadisticamente significativo se puede inferir que posiblemente haya problemas de informacion relevante omitida es decir el termino del error esta correlacionado con las variables exogenas.

# EJERCICIO 2

* El parámetro $\beta$ puede ser calculado con álgebra matricial y el paquete numpy

* La ecuacón lineal que queremos estimar es: $ y = X\beta + u $

* Para resolver la ecuación para el parámetro $ \beta $,   minimizamos la suma del cuadrado de los residuales. $\underset{\hat{\beta}}{\min} \hat{u}'\hat{u}$

* Rearreglando la primera ecuación y substituyendo en la segunda ecuación podemo escribir:$\underset{\hat{\beta}}{\min} \ (Y - X\hat{\beta})' (Y - X\hat{\beta})$

* Resolviendo este problema de optimización obtenemos la solución para los coeficientes $ \hat{\beta} $ 
$$\hat{\beta} = (X'X)^{-1}X'y
$$
* Utilizando la información ya descrita, calcule  $ \hat{\beta} $ del modelo 1 utilizando numpy - 
Sus resultados deberían ser los mismos que los resultados obtenidos en la clase de la semana pasada al correr:

   **X1 = ['const', 'avexpr']**

   **reg1 = sm.OLS(df2['logpgp95'], df2[X1], missing='drop').fit()**}
   
##### *Para  realizar este ejercicio utiliza los datos **'maketable1.dta'**

In [21]:
# Importar datos
df1 = pd.read_stata('maketable1.dta')

# Borre los NA's
df1_subset1 = df1.dropna(subset = ['logpgp95', 'avexpr'])
df1_subset1


,shortnam,euro1900,excolony,avexpr,logpgp95,cons1,cons90,democ00a,cons00a,extmort4,logem4,loghjypl,baseco
1,AGO,8.000000,1.0,5.363636,7.770645,3.0,3.0,0.0,1.0,280.000000,5.634789,-3.411248,1.0
2,ARE,0.000000,1.0,7.181818,9.804219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ARG,60.000004,1.0,6.386364,9.133459,1.0,6.0,3.0,3.0,68.900002,4.232656,-0.872274,1.0
5,AUS,98.000000,1.0,9.318182,9.897972,7.0,7.0,10.0,7.0,8.550000,2.145931,-0.170788,1.0
6,AUT,100.000000,0.0,9.727273,9.974877,NaN,NaN,NaN,NaN,NaN,NaN,-0.343900,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,YEM,0.000000,1.0,6.363636,6.646390,NaN,NaN,NaN,NaN,NaN,NaN,-1.551169,NaN
159,ZAF,22.000000,1.0,6.863636,8.885994,3.0,7.0,3.0,3.0,15.500000,2.740840,-1.386294,1.0
160,ZAR,8.000000,1.0,3.500000,6.866933,1.0,1.0,0.0,1.0,240.000000,5.480639,-3.411248,1.0
161,ZMB,3.000000,1.0,6.636364,6.813445,3.0,1.0,0.0,1.0,NaN,NaN,-2.975930,NaN


###### Añade la constante

In [24]:
df1_subset1.insert(13,'const',1)
#df1_subset1['const']=1
df1_subset1.head()

ValueError: cannot insert const, already exists

##### Define las variables X y Y

In [25]:
#
y = np.asarray(df1_subset1['logpgp95'])

X = np.asarray(df1_subset1[['const',  'avexpr']])

#### Calcula Beta

In [26]:
β_hat = np.linalg.solve((X.T @ X).T ,X.T @ y)

In [27]:
β_hat = np.linalg.solve((X.T @ X).T ,X.T @ y)
test= np.linalg.inv(X.T @ X)@X.T@y

#### Imprime los resultados, ¿son similares sus resultados?



In [28]:

print(f'β_0 = {β_hat[0]:.4}')
print(f'β_1 = {β_hat[1]:.4}')

β_0 = 4.626
β_1 = 0.5319


In [29]:

print(f'β_0 = {test[0]:.4}')
print(f'β_1 = {test[1]:.4}')

β_0 = 4.626
β_1 = 0.5319
